# Objectif 
Construire un mdèle de reconnaissence d'émotion depuis la parole.


Un autre dépôt contient un modèle de détection d'émotions à partir des images et des vidéos.

# Librairies

In [182]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn import preprocessing

# Préparation des données 


La base de données utilisé est extraite de RAVDESS (Ryerson Audio-Visual Database of Emotional Speech and Song) dataset.
Téléchargable gratuitement du lien suivant : https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view


## Caractéristiques et variables (Features)

Les variables sur lequel nos modèles vont se baser pour détecter l'émotion seront les suivants :

   __chroma__ : Concerne les 12 différentes classes de ton

   **mfcc** : Mel Frequency Cepstral Coefficients
        
   **mel** : Mel Spectrogram Frequency

On utilisera la librairie **librosa** pour extraire ces variables.

Notre fonction prend en entrée des booleans indiquant, pour chaque variable, si on la prend comme feature ou pas.

Pour plus de détails sur la signification : http://themarvinproject.free.fr/final/node4.html

In [183]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

Nous n'allons garder que quatres classes parmi les 8 déjà présents

In [184]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

## Chargement des données

La fonction suivante prend en paramètre le pourcentage des données en test et renvoie sous forme d'arrays
les données d'entrainement et de test

Pour l'entrainement d'un modèle de réseaux de neurones nous seront amené à encoder nos labels 

In [185]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

#Encoding labels
le = preprocessing.LabelBinarizer()
le.fit(y_train)

LabelBinarizer()

In [186]:
#DataFlair - Get the shape of the training and testing datasets
print('Training samples : ',x_train.shape[0],'; Test samples : ', x_test.shape[0])
print('number of features : ',x_train.shape[1])

Training samples :  576 ; Test samples :  192
number of features :  180


# Models

On va essayer de tester deux modèles dans ce notebook.

Le premier est celui de MLPClassifier de Keras qui utilise un réseaux de neurones interne et le deuxième est un simple modèle de réseaux de neurones.

In [ ]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
MLP=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Full neural network model
Fnn = Sequential()
Fnn.add(Dense(300,input_dim=180,activation='relu'))
Fnn.add(Dense(128,activation='relu'))
Fnn.add(Dense(4,activation='softmax'))

Fnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])



# Entrainement des modèles

In [188]:
MLP.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
Fnn.fit(x_train,le.transform(y_train),epochs=100)

# Tester les modèles

Dans un premier temps on va tester les deux modèles sur
un seul ensemble de données

In [191]:
#Predict for the test set using MLP model
y_pred=MLP.predict(x_test)
print("MLP classification report")
print(classification_report(y_pred,y_test))

#Test the FNN model
y_pred=le.inverse_transform(Fnn.predict(x_test))
print("FNN classification report")
print(classification_report(y_pred,y_test))

MLP classification report
              precision    recall  f1-score   support

        calm       0.83      0.94      0.88        47
     disgust       0.87      0.61      0.71        66
     fearful       0.76      0.71      0.73        52
       happy       0.43      0.70      0.54        27

    accuracy                           0.73       192
   macro avg       0.72      0.74      0.72       192
weighted avg       0.77      0.73      0.73       192

FNN classification report
              precision    recall  f1-score   support

        calm       0.91      0.96      0.93        50
     disgust       0.87      0.68      0.76        59
     fearful       0.86      0.71      0.78        59
       happy       0.45      0.83      0.59        24

    accuracy                           0.78       192
   macro avg       0.77      0.80      0.76       192
weighted avg       0.82      0.78      0.79       192



# Conclusion
On peut voir que le modèle de réseaux de neurones donne des résultats meuilleurs que MLPClassifier .

Pour confirmer cette conclusion on applique une validation croisée.

In [167]:
scores = cross_val_score(MLP,np.vstack((x_train,x_test)),y_train+y_test,cv=5)
print("accuracy : ",scores*100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(scores*100), np.std(scores*100)))

accuracy :  [67.53246753 70.77922078 68.18181818 81.04575163 72.54901961]
72.02% (+/- 4.86%)


In [162]:
from sklearn.model_selection import StratifiedKFold

seed = 7
np.random.seed(seed)

X = np.vstack((x_train,x_test))
Y = np.array(y_train+y_test)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
    Fnn = Sequential()
    Fnn.add(Dense(300,input_dim=180,activation='relu'))
    Fnn.add(Dense(128,activation='relu'))
    Fnn.add(Dense(4,activation='softmax'))
    Fnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    Fnn.fit(X[train], le.transform(Y[train]),batch_size=10, verbose=0,epochs=100)
    # evaluate the model
    scores = Fnn.evaluate(X[test], le.transform(Y[test]), verbose=0)
    print("%s: %.2f%%" % (Fnn.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

accuracy: 77.27%
accuracy: 77.27%
accuracy: 74.03%
accuracy: 78.43%
accuracy: 80.39%
77.48% (+/- 2.07%)
